In [1]:
# doing some imports
import json
import pickle

In [2]:
# bringing in the list of the answers and the 
# the questions from the squad list
with open("pred_ans_list", mode="rb") as f:
    pred_ans_list = pickle.load(f,)

In [3]:
print(len(pred_ans_list))
pred_ans_list[:3]

10000


['late1990s', 'singinganddancing', '2003']